In [22]:
import requests
import pandas as pd
import os
import ast

# Function to call Namsor API and save the response as a file
def call_namsor_api_and_save(first_name, last_name):
    api_url = "https://v2.namsor.com/NamSorAPIv2/api2/json/diasporaBatch"
    api_key = "0741d3c396c83c8761dd49cb51760a3f"  # Replace with your Namsor API key

    headers = {
        "X-API-KEY": api_key,
    }

    payload = {
      "personalNames": [
        {
          "id": "0d7d6417-0bbb-4205-951d-b3473f605b56",
          "firstName": first_name,
          "lastName": last_name,
          "countryIso2": "US"
        }
      ]
    }


    try:
        response = requests.post(api_url, headers=headers, json=payload)
        response.raise_for_status()  # Raise an exception for non-2xx status codes
        result = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Failed to call API for name: {name}. Error: {e}")
        result = {
            "ethnicity": "",
            "ethnicityAlt": "",
        }
        first_name, last_name = "", ""  # Set default empty values for first_name and last_name in case of API call failure

    output_file = f"output/{first_name}_{last_name}.txt"
    with open(output_file, "w") as file:
        file.write(str(result))

    return result

In [23]:
# Iterate through each file in the directory
def extract_ethnicity_data(first_name, last_name):
    directory_path = "output"
    ethnicities = []
    alternate_ethnicities = []
    first_names = []
    last_names = []
    # Iterate through each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r') as file:
                # Use ast.literal_eval to parse the JSON data with single quotes
                data = ast.literal_eval(file.read())
                personal_names = data.get('personalNames', [])
                if personal_names:
                    for name_data in personal_names:
                        first_name = name_data.get('firstName', '')
                        last_name = name_data.get('lastName', '')
                        ethnicity = name_data.get('ethnicity', '')
                        alternate_ethnicity = name_data.get('ethnicityAlt', '')

                        first_names.append(first_name)
                        last_names.append(last_name)
                        ethnicities.append(ethnicity)
                        alternate_ethnicities.append(alternate_ethnicity)

    # Create a DataFrame from the extracted data
    df = pd.DataFrame({
        'First Name': first_names,
        'Last Name': last_names,
        'Ethnicity': ethnicities,
        'Alternate Ethnicity': alternate_ethnicities
    })

    # Save the DataFrame to an Excel file
    output_excel_file = "C:/Users/ParthSharma/Downloads/List.xlsx"  # Replace with the desired output file path
    df.to_excel(output_excel_file, index=False)


In [24]:
# Load the names from the Excel file
excel_file_path = "C:/Users/ParthSharma/Downloads/Names_List.xlsx"
df = pd.read_excel(excel_file_path, sheet_name="Sheet3")

# Call the Namsor API and extract ethnicity function to get the names and ethinicity
df["Full Name"] = df["First Name"] + " " + df["Last Name"]
df["Ethnicity"] = ""
df["Alternate Ethnicity"] = ""
first_name_column = "First Name"
last_name_column = "Last Name"
for index, row in df.iterrows():
    first_name = row[first_name_column]
    last_name = row[last_name_column]
    result = call_namsor_api_and_save(first_name, last_name)
    extract_ethnicity_data(first_name, last_name)
